<h1><a>Lec 02 NLP基础与扫盲</a></h1>

本节以NLTK为基础配合讲解自然语言处理的原理。

## 目录：
- <a href='#nltk'>NLTK</a>
- <a href='#textprocessing'>文本处理流程</a>
    - <a href='#tokenize'>分词</a>
    - <a href='#standard'>归一化</a>
    - <a href='#stopwords'>停止词</a>
- <a href='#3examples'>NLP经典三案例</a>
    - <a href='#sentiment'>情感分析</a>
    - <a href='#similarity'>文本相似度</a>
    - <a href='#classification'>文本分类</a>
- <a href='#deeplearning'>深度学习加持</a>
    - <a herf='#autoencoder'>Autoencoder</a>
    - <a herf='#word2vec'>Word2Vec</a>

----

<h2><a name='nltk'>1.NLTK</a></h2>

官网：http://www.nltk.org/

Python上著名的自然语言处理库。自带语料库、词性分类库、自带分类、分词，等等功能强大的社区支持，还有N多的简单版wrapper。

In [ ]:
# 安装语料库
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


### NLTK功能一览表
|NLTK Modules|Functionality|
|:-|:-:|
|nltk.corpus|Corpus|
|nltk.tokenize, nltk.stem| Tokenizers, stemmers|
|nltk.collocations|t-test, chi-squared, mutual-info|
|nltk.tag|n-gram, backoff, Brill, HMM, TnT|
|nltk.classify, nltk.cluster| Decision tree, Naive Bayes, K-means|
|nltk.chunk| Regex, n-gram, named entity|
|nltk.parsing| Parsing|
|nltk.sem, nltk.interence| Semantic interpretation|
|nltk.metrics| Evaluation metrics|
|nltk.probability| Probability & Estimation|
|nltk.app, nltk.chat| Applications|

### NLTK自带语料库

In [1]:
from nltk.corpus import brown
brown.categories()

['adventure',
 'belles_lettres',
 'editorial',
 'fiction',
 'government',
 'hobbies',
 'humor',
 'learned',
 'lore',
 'mystery',
 'news',
 'religion',
 'reviews',
 'romance',
 'science_fiction']

In [2]:
len(brown.sents())

57340

In [3]:
len(brown.words())

1161192

<h2><a name='textprocessing'>2.文本处理流程</a><h2>

<img src='./images/nltk01.png' width='70%'/>

<h3><a name='tokenize'>2.1 分词</a></h3>

Tokenize

将长句子拆成有“意义”的小部件。

In [4]:
import nltk
sentence = 'hello, world!'
tokens = nltk.word_tokenize(sentence)
tokens

['hello', ',', 'world', '!']

#### 中英文NLP的区别

<img src='./images/nltk02.png' width='70%'/>
<img src='./images/nltk03.png' width='30%'/>

In [5]:
# 中文分词
import jieba
seg_list = jieba.cut('我来到北京清华大学', cut_all=True) # 全模式
print('Full Mode:', '/'.join(seg_list))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/w2/qnnfb62x2g760j3nkh9q4ptr0000gn/T/jieba.cache
Loading model cost 0.733 seconds.
Prefix dict has been built succesfully.


Full Mode: 我/来到/北京/清华/清华大学/华大/大学


In [6]:
seg_list = jieba.cut('我来到北京清华大学', cut_all=False) # 精确模式
print('Default Mode:', '/'.join(seg_list))

Default Mode: 我/来到/北京/清华大学


seg_list = jieba.cut('他来到了网易杭研大厦') # 默认是精确模式
print('Default Mode:', ','.join(seg_list))

In [10]:
seg_list = jieba.cut_for_search('小明硕士毕业于中国科学院计算所，后在日本京都大学深造') # 搜索引擎模式
print(','.join(seg_list))

小明,硕士,毕业,于,中国,科学,学院,科学院,中国科学院,计算,计算所,，,后,在,日本,京都,大学,日本京都大学,深造


#### 分词之后的效果
<img src='./images/nltk04.png' width='50%'/>

#### 有时候tokenize没那么简单
比如：  
社交网络上，这些乱七八糟的不和语法不合逻辑的语言很多：   
拯救@某人，表情符号，URL，#话题符号

<img src='./images/nltk05.png' width='50%'/>

In [11]:
# 社交网络语言的tokenize
from nltk.tokenize import word_tokenize

tweet = 'RT @angelababy: love you baby! :D http://ah.love #168cm'
print(word_tokenize(tweet))

['RT', '@', 'angelababy', ':', 'love', 'you', 'baby', '!', ':', 'D', 'http', ':', '//ah.love', '#', '168cm']


#### 对社交网络语言的tokenize，在预处理过程中，用re正则表达式预处理。

#### 正则表达式
对照表：
http://www.regexlab.com/zh/regref.htm

In [13]:
import re
emoticons_str = r'''
(?:
    [:=;] #眼睛
    [oO\-]? # ⿐鼻⼦子
    [D\)\]\(\]/\\OpP] # 嘴
)
'''
regex_str = [emoticons_str, 
            r'<[^>]+>', # HTML tags
            r'(?:@[\w_]+)', # @某⼈人
            r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # 话题标签
            r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',
            # URLs
            r'(?:(?:\d+,?)+(?:\.?\d+)?)', # 数字
            r"(?:[a-z][a-z'\-_]+[a-z])", # 含有 - 和 ‘ 的单词
            r'(?:[\w_]+)', # 其他
            r'(?:\S)' # 其他
            ]

In [31]:
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

In [35]:
def tokenize(s):
    return tokens_re.findall(s)

In [36]:
def preprocess(s, lowercase=False):
    def tokenize(s):
        return tokens_re.findall(s)
    tokens = tokenize(s)
    if lowercase:
        tokenize = [token if emotion_re.search(token) else token.lower() for token in tokens]
    return tokens

In [37]:
tweet = 'RT @angelababy: love you baby! :D http://ah.love #168cm'
print(preprocess(tweet))

['RT', '@angelababy', ':', 'love', 'you', 'baby', '!', ':D', 'http://ah.love', '#168cm']


<h3><a name='standard'>2.2 归一化</a></h3>

- 纷繁复杂的词形

<img src='./images/nltk06.png' width='50%'/>
<img src='./images/nltk07.png' width='70%'/>

#### 词形归一化
- Stemming 词干提取：一般来说，就是把不影响词形的inflection的小尾巴砍掉
    - walking 砍掉ing => walk
    - walked 砍掉ed => walk
    
- Lemmatization 词形归一：把各种类型的词的变形，都归为一个形式
    - went 归一 => go
    - are 归一 => be

#### NLTK实现Stemming：nltk.stem.porter.PorterStemmer

In [38]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
porter_stemmer.stem('maximum')

'maximum'

In [39]:
porter_stemmer.stem('presumably')

'presum'

In [40]:
porter_stemmer.stem('multiply')

'multipli'

In [41]:
porter_stemmer.stem('provision')

'provis'

In [45]:
porter_stemmer.stem('went')

'went'

In [46]:
porter_stemmer.stem('wenting')

'went'

#### NLTK实现Stemming：nltk.stem.SnowballStemmer

In [43]:
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer('english')
snowball_stemmer.stem('maximum')

'maximum'

In [44]:
snowball_stemmer.stem('presumably')

'presum'

#### NLTK实现Stemming：nltk.stem.lancaster.LancasterStemmer

In [47]:
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
lancaster_stemmer.stem('maximum')

'maxim'

In [48]:
lancaster_stemmer.stem('presumably')

'presum'

In [49]:
lancaster_stemmer.stem('presumably')

'presum'

#### NLTK实现Lemma: nltk.stem.WordNetLemmatizer

In [50]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
wordnet_lemmatizer.lemmatize('dogs')

'dog'

In [51]:
wordnet_lemmatizer.lemmatize('churches')

'church'

In [52]:
wordnet_lemmatizer.lemmatize('aardwolves')

'aardwolf'

In [53]:
wordnet_lemmatizer.lemmatize('abaci')

'abacus'

In [54]:
wordnet_lemmatizer.lemmatize('hardrock')

'hardrock'

#### Lemma的小问题
<img src='./images/nltk08.png' width='40%'/>

#### NLTK更好地实现Lemma

In [56]:
# 没有PosTag词性标注，默认是 NN(名词)
wordnet_lemmatizer.lemmatize('are')

'are'

In [57]:
wordnet_lemmatizer.lemmatize('is')

'is'

In [58]:
# 加上Pos Tag（词性标注）
wordnet_lemmatizer.lemmatize('is', pos='v')

'be'

In [59]:
wordnet_lemmatizer.lemmatize('are', pos='v')

'be'

#### 词形标注Part-Of-Speech
<img src='./images/nltk09.png' width='80%'/>

In [61]:
# NLTK的词性标注 POS Tag
import nltk
text = nltk.word_tokenize('what does the fox say') # 先分词
print(text)
print(nltk.pos_tag(text)) # 后POS Tag

['what', 'does', 'the', 'fox', 'say']
[('what', 'WDT'), ('does', 'VBZ'), ('the', 'DT'), ('fox', 'NNS'), ('say', 'VBP')]


<h3><a name='stopwords'>2.3 停止词</a></h3>

一千个he，有一千种指代。    
一千个the，有一千种指事。

对于注重理解文本【意思】的应用场景来说，歧义太多。

全体stopwords列表：http://www.ranks.nl/stopwords

#### NLTK去除stopwords
首先，记得在console里面加载一下词库，或者，nltk.download('stopwords')

In [65]:
from nltk.corpus import stopwords
# 先tokenize分词，得到word_list
word_list = nltk.word_tokenize('I am chinese.')
# 然后filter一把
filtered_words = [word for word in word_list if word not in stopwords.words('english')]
print(filtered_words)

['I', 'chinese', '.']


----

<h3>NLTK文本处理总结：一条typical的文本预处理流水线</h3>

<img src='./images/nltk10.png' width='40%'/>

<h2><a name='3examples'>3.NLP经典三案例</a></h2>

什么是自然语言处理？

<img src='./images/nltk11.png' width='40%'/>

文本预处理让我们得到了什么？
<img src='./images/nltk12.png' width='40%'/>

NLTK在NLP上的经典应用：情感分析、文本相似度、文本分类。

<h3><a name='sentiment'>3.1 情感分析</a></h3>

<img src='./images/nltk13.png' width='60%'/>

最简单的sentiment dictionary:
- like 1
- good 2
- bad -2
- terrible -3

类似于关键词打分机制

比如：AFINN-111
http://www2.imm.dtu.dk/pubdb/views/publication_details.php?id=6010

<h3><a name='similarity'>3.2 文本相似度</a></h3>

<h3><a name='classification'>3.3 文本分类</a></h3>

<h2><a name='deeplearning'>4.深度学习加持</a></h2>

<h3><a name='autoencoder'>4.1 Autoencoder</a></h3>

<h3><a name='word2vec'>4.2 Word2Vec</a></h3>